In [ ]:
#pip install pyodbc

In [2]:
import pyodbc
import pandas as pd

In [ ]:
# Database connection parameters
server = 'PT-L163255\SQLEXPRESS01'  # e.g., 'localhost' or 'server_name'
database = 'University'  # e.g., 'my_database'

# Connection string
conn_str = (
    f"DRIVER={{ODBC Driver 17 for SQL Server}};"
    f"SERVER={server};"
    f"DATABASE={database};"
    f'Trusted_Connection=yes;'
)

# Connect to the SQL Server database
conn = pyodbc.connect(conn_str)

# Execute a query and load data into a DataFrame
#query = "SELECT * FROM aluno"  # Replace 'your_table' with your actual table name
#df = pd.read_sql_query(query, conn)

query = f"SELECT * FROM aluno"
cursor = conn.cursor()
cursor.execute(query)
results = [list(row) for row in cursor.fetchall()]
colunas = ['nr_aluno', 'nome', 'apelido', 'curso', 'media_entrada', 'curso_concluido', 'media_final']
df = pd.DataFrame(results, columns= colunas )

In [38]:
df = pd.read_csv('/workspaces/Data_Engineering/output_dataengineering.csv')
df

,0,nr_aluno,nome,apelido,curso,media_entrada,curso_concluido,media_final
0,0,20031358,Leonardo,Pires,Psicologia,15,True,16.0
1,1,2022158,Raquel,Laranjeira,Informática,17,True,11.0
2,2,2003812,Anne,Silva,Informática,16,True,16.0
3,3,2022901,Patrícia,Castelo-Branco,Informática,11,True,17.0
4,4,2019303,Avelino,Sampaio,Comunicação,13,True,18.0
...,...,...,...,...,...,...,...,...
1755,1755,2019869,Alice,Laranjeira,Informática,11,True,17.0
1756,1756,20221093,Patrícia,Pereira,Comunicação,14,True,16.0
1757,1757,20071160,Lucas,Capicua,Informática,12,True,20.0
1758,1758,2020288,Pedro,Capicua,Comunicação,13,True,19.0


### Check Duplicados

In [39]:
#Não há duplicados
duplicados = df['nr_aluno'].duplicated()
duplicados.sum()

np.int64(0)

### Collect all 'Apelido' that contains '()'

In [40]:
df_apelido = df[df['apelido'].str.contains(r'\(.*\)')]
df_apelido

,0,nr_aluno,nome,apelido,curso,media_entrada,curso_concluido,media_final
100,100,2210000,Thomas Anderson,(Neo),Informática,20,True,20.0
201,201,2290110,Anakin Skywalker,(Lord Vader),Psicologia,20,True,20.0
804,804,2210030,Sininho,(Terra do Nunca),Cintilar,-20,True,-20.0


### Select that are not in the list [['Psicologia', 'Informática', 'Comunicação']]

In [41]:
cursos = ['Psicologia', 'Informática', 'Comunicação']
for i in df['curso']:
    if i not in cursos:
        print(i)

nan
nan
Desconhecido
Medicina Veterinária
nan
Medicina Veterinária
nan
Medicina Veterinária
Medicina Veterinária
Desconhecido
Má Vida
Cintilar


In [44]:
df['curso'] = df['curso'].apply(lambda x: x if x in cursos else 'NULL')
df

,0,nr_aluno,nome,apelido,curso,media_entrada,curso_concluido,media_final
0,0,20031358,Leonardo,Pires,Psicologia,15,True,16.0
1,1,2022158,Raquel,Laranjeira,Informática,17,True,11.0
2,2,2003812,Anne,Silva,Informática,16,True,16.0
3,3,2022901,Patrícia,Castelo-Branco,Informática,11,True,17.0
4,4,2019303,Avelino,Sampaio,Comunicação,13,True,18.0
...,...,...,...,...,...,...,...,...
1755,1755,2019869,Alice,Laranjeira,Informática,11,True,17.0
1756,1756,20221093,Patrícia,Pereira,Comunicação,14,True,16.0
1757,1757,20071160,Lucas,Capicua,Informática,12,True,20.0
1758,1758,2020288,Pedro,Capicua,Comunicação,13,True,19.0


In [56]:
df['media_entrada'] = pd.to_numeric(df['media_entrada'], errors='coerce')
filtered_df_entrada= df[(df['media_entrada'] < 0) | (df['media_entrada'] > 20) | (df['media_entrada'].isnull())]
filtered_df_entrada

,0,nr_aluno,nome,apelido,curso,media_entrada,curso_concluido,media_final
34,34,2019990,Leonardo,Capicua,Comunicação,22,True,12.0
42,42,2021652,Lucas,Machado,Informática,21,True,17.0
56,56,20201386,Pedro,Pires,Psicologia,22,True,18.0
79,79,20201046,Miguel,Silva,Psicologia,-1,True,17.0
85,85,2019514,Diana,Sampaio,Informática,-2,True,12.0
804,804,2210030,Sininho,(Terra do Nunca),NULL,-20,True,-20.0


In [66]:
df['media_final'] = pd.to_numeric(df['media_final'], errors='coerce')
filtered_df_final = df[(df['media_final'] < 0) | (df['media_entrada'] > 20) | (df['media_final'].isnull())]
filtered_df_final

,0,nr_aluno,nome,apelido,curso,media_entrada,curso_concluido,media_final
25,25,2003821,Nuno,Laranjeira,Psicologia,16,False,NaN
27,27,200739,Alice,Pires,Psicologia,17,False,NaN
30,30,2003485,Alice,Pires,Psicologia,13,False,NaN
33,33,20211052,Raquel,Castelo,Comunicação,16,False,NaN
34,34,2019990,Leonardo,Capicua,Comunicação,22,True,12.0
42,42,2021652,Lucas,Machado,Informática,21,True,17.0
48,48,202245,Rui,Pires,Informática,10,True,-2.0
52,52,2007469,Nuno,da Silva,Psicologia,14,True,-1.0
55,55,20071679,Jorge,Sampaio,Comunicação,10,True,-1.0
56,56,20201386,Pedro,Pires,Psicologia,22,True,18.0


In [69]:
error_dataframe = pd.concat([df_apelido, filtered_df_entrada ,filtered_df_final], ignore_index=True)
df_no_duplicates =  error_dataframe.drop_duplicates()

In [70]:
error_dataframe = pd.concat([df_apelido, filtered_df_entrada ,filtered_df_final], ignore_index=True)
df_no_duplicates =  error_dataframe.drop_duplicates()
#df_no_duplicates_reset = df_no_duplicates.reset_index(drop=True)

df_no_duplicates


,0,nr_aluno,nome,apelido,curso,media_entrada,curso_concluido,media_final
0,100,2210000,Thomas Anderson,(Neo),Informática,20,True,20.0
1,201,2290110,Anakin Skywalker,(Lord Vader),Psicologia,20,True,20.0
2,804,2210030,Sininho,(Terra do Nunca),Cintilar,-20,True,-20.0
3,34,2019990,Leonardo,Capicua,Comunicação,22,True,12.0
4,42,2021652,Lucas,Machado,Informática,21,True,17.0
5,56,20201386,Pedro,Pires,Psicologia,22,True,18.0
6,79,20201046,Miguel,Silva,Psicologia,-1,True,17.0
7,85,2019514,Diana,Sampaio,Informática,-2,True,12.0
8,804,2210030,Sininho,(Terra do Nunca),NULL,-20,True,-20.0
9,25,2003821,Nuno,Laranjeira,Psicologia,16,False,NaN


In [71]:
# Convert DataFrame to a text file
error_dataframe.to_csv('fase_1.txt', sep='\t', index=False)


### DataFrame with correct Data

In [72]:
for col in df.columns:
    if col in error_dataframe.columns:
        df[col] = df[col].astype(error_dataframe[col].dtype)

In [73]:
merged_df = df.merge(error_dataframe, how='left', indicator=True)
df_valid_values = merged_df[merged_df['_merge'] == 'left_only'].drop(columns=['_merge'])
df_valid_values.reset_index(drop=True, inplace=True)
df_valid_values

,0,nr_aluno,nome,apelido,curso,media_entrada,curso_concluido,media_final
0,0,20031358,Leonardo,Pires,Psicologia,15,True,16.0
1,1,2022158,Raquel,Laranjeira,Informática,17,True,11.0
2,2,2003812,Anne,Silva,Informática,16,True,16.0
3,3,2022901,Patrícia,Castelo-Branco,Informática,11,True,17.0
4,4,2019303,Avelino,Sampaio,Comunicação,13,True,18.0
...,...,...,...,...,...,...,...,...
1724,1755,2019869,Alice,Laranjeira,Informática,11,True,17.0
1725,1756,20221093,Patrícia,Pereira,Comunicação,14,True,16.0
1726,1757,20071160,Lucas,Capicua,Informática,12,True,20.0
1727,1758,2020288,Pedro,Capicua,Comunicação,13,True,19.0


In [74]:
df_valid_values.to_csv('fase_1.xlsx', index=False)

In [ ]:
conn.close()